In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! git clone https://github.com/broccubali/autisticadventures

Cloning into 'autisticadventures'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 67 (delta 25), reused 15 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 3.71 MiB | 9.39 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
import os
from torch.utils.data import Dataset
import torch
import numpy as np


class CC200Data(Dataset):
    def __init__(self, path, mapping, labels):
        super().__init__()
        self.path = path
        self.mapping = mapping
        self.folder = os.listdir(self.path)
        self.labels = self._map_labels(labels)
        self.files = [np.loadtxt(f"{path}/{i}") for i in self.folder]
        self.region_indices = self._region_mapping()
        self.region_coeff = torch.tensor([self._region_coeffs(i) for i in self.files], dtype=torch.float32)
        self.subnetwork_coeff = [self._subnetwork_coeffs(i) for i in self.files]
        self.region_start_indices = {}
        for i in range(1, 8):
            if i == 1:
                self.region_start_indices[i] = 0
            else:
                self.region_start_indices[i] = self.region_start_indices[i - 1] + len(
                    self.region_indices[i - 1]
                )

    def _map_labels(self, mapping):
        labels = []
        for i in self.folder:
            if i[:-14] in mapping.keys():
                labels.append(mapping[i[:-14]])
            else:
                print(i)

        return torch.tensor(labels, dtype=torch.long)

    def _region_coeffs(self, x):
        b = np.zeros_like(x)
        y = 0
        for i in self.region_indices:
            b[:, y : y + len(self.region_indices[i])] = x[:, self.region_indices[i]]
            y += len(self.region_indices[i])
        b = b[:, 15:]
        return np.corrcoef(b, rowvar=False)

    def _region_mapping(self):
        region_indices = {i: [] for i in range(0, 8)}
        for i, j in self.mapping.items():
            region_indices[j].append(i - 1)
        return region_indices

    def _subnetwork_coeffs(self, x):
        correlation_matrices = []

        for file_idx, (region_id, indices) in enumerate(self.region_indices.items()):
            if not indices:
                print(f"[INFO] Region {region_id} has no indices, skipping.")
                continue

            submatrix = x[:, indices]
            std = np.std(submatrix, axis=0)

            zero_std_count = np.sum(std == 0)
            if zero_std_count > 0:
                print(f"[INFO] File {self.folder[file_idx]} - Region {region_id} has {zero_std_count} constant columns.")

            try:
                correlation_matrix = np.corrcoef(submatrix, rowvar=False)
            except Exception as e:
                print(f"[ERROR] Correlation failed in region {region_id}")
                print(f"Exception: {e}")
                continue

            flat_corr = self._flatten_matrix(correlation_matrix)
            correlation_matrices.append(torch.tensor(flat_corr, dtype=torch.float32))

        return correlation_matrices

    def _flatten_matrix(self, matrix):
        idx = np.triu_indices_from(matrix, k=1)
        return matrix[idx]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        region_corr = self.region_coeff[idx]
        subnetwork_corrs = self.subnetwork_coeff[idx]
        labels = self.labels[idx]
        return region_corr, subnetwork_corrs, labels

In [4]:
!wget https://raw.githubusercontent.com/broccubali/AutisticAdventures/main/cc200_to_yeo7_mapping.csv
!wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv

--2025-04-18 07:43:48--  https://raw.githubusercontent.com/broccubali/AutisticAdventures/main/cc200_to_yeo7_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319 (1.3K) [text/plain]
Saving to: ‘cc200_to_yeo7_mapping.csv’

cc200_to_yeo7_mappi 100%[===================>]   1.29K  --.-KB/s    in 0s      

2025-04-18 07:43:49 (55.4 MB/s) - ‘cc200_to_yeo7_mapping.csv’ saved [1319/1319]

--2025-04-18 07:43:49--  https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.232.16, 52.217.167.0, 52.217.101.158, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.232.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449443 (439K

In [5]:
import pandas as pd

df = pd.read_csv('autisticadventures/cc200_to_yeo7_mapping.csv')
cc200_to_yeo7_mapping = dict(zip(df['CC200_Region'], df['Yeo7_Network'])) 

In [6]:
df = pd.read_csv("Phenotypic_V1_0b_preprocessed1.csv")
df = df[["FILE_ID", "DX_GROUP"]]
labels_mapping = dict(zip(df["FILE_ID"], df["DX_GROUP"]))

In [7]:
# had to fix this here cuz pytorch cross entropy loss needs 0 and 1 not 1 and 2
new_labels_mapping = {}
for key, value in labels_mapping.items():
    new_labels_mapping[key] = value - 1  # Subtract 1 to convert 1,2 to 0,1

# Recreate your dataset with adjusted labels
dataset = CC200Data("/kaggle/input/autistic-brains/Outputs/cpac/nofilt_noglobal/rois_cc200", cc200_to_yeo7_mapping, new_labels_mapping)

/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/tmp/ipykernel_31/1401220553.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  self.region_coeff = torch.tensor([self._region_coeffs(i) for i in self.files], dtype=torch.float32)


[INFO] File UM_1_0050315_rois_cc200.1D - Region 0 has 2 constant columns.
[INFO] File MaxMun_c_0051340_rois_cc200.1D - Region 5 has 1 constant columns.
[INFO] File MaxMun_c_0051340_rois_cc200.1D - Region 5 has 1 constant columns.
[INFO] File MaxMun_c_0051340_rois_cc200.1D - Region 5 has 3 constant columns.
[INFO] File UM_1_0050315_rois_cc200.1D - Region 0 has 1 constant columns.
[INFO] File NYU_0051123_rois_cc200.1D - Region 1 has 19 constant columns.
[INFO] File Pitt_0050045_rois_cc200.1D - Region 2 has 13 constant columns.
[INFO] File USM_0050443_rois_cc200.1D - Region 3 has 16 constant columns.
[INFO] File Stanford_0051165_rois_cc200.1D - Region 4 has 9 constant columns.
[INFO] File NYU_0050993_rois_cc200.1D - Region 6 has 7 constant columns.
[INFO] File UCLA_1_0051211_rois_cc200.1D - Region 7 has 15 constant columns.
[INFO] File MaxMun_c_0051340_rois_cc200.1D - Region 5 has 1 constant columns.
[INFO] File NYU_0050993_rois_cc200.1D - Region 6 has 1 constant columns.
[INFO] File UM_1

In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [9]:
unique_labels = set()
for _, _, labels in train_loader:
    unique_labels.update(labels.numpy())
print(f"updated label values: {sorted(list(unique_labels))}")

updated label values: [0, 1]


In [10]:
shapes = []
a = next(iter(dataset))[1]
for i in a:
    shapes.append(i.shape[0])

# len(shapes)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MHSA(nn.Module):
    def __init__(self, embd_dim, num_heads):
        super().__init__()
        self.embd_dim = embd_dim
        self.num_heads = num_heads
        self.head_size = self.embd_dim // self.num_heads
        self.q = nn.Linear(self.embd_dim, self.embd_dim)
        self.k = nn.Linear(self.embd_dim, self.embd_dim)
        self.v = nn.Linear(self.embd_dim, self.embd_dim)
        self.d = self.head_size ** 0.5
        self.mlp = nn.Linear(self.embd_dim, self.embd_dim)
        self.layer_norm = nn.LayerNorm(self.embd_dim)  
        
    def forward(self, x):
        batch_size, M, _ = x.shape
        norm = self.layer_norm(x)
        q = self.q(norm).view(batch_size, M, self.num_heads, self.head_size).transpose(1, 2)
        k = self.k(norm).view(batch_size, M, self.num_heads, self.head_size).transpose(1, 2)
        v = self.v(norm).view(batch_size, M, self.num_heads, self.head_size).transpose(1, 2)
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / self.d
        attn_scores = attn_scores.masked_fill(torch.eye(M, device=x.device).bool(), float('-inf'))
        attn_weights = F.softmax(attn_scores, dim=-1)
        context = torch.matmul(attn_weights, v).transpose(1, 2).reshape(batch_size, M, self.embd_dim)
        out = self.mlp(context)
        return out + x, attn_weights

In [12]:
class SubnetworkEmbedder(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.mlp(x)

In [13]:
class RegionEmbedder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.region_conv = nn.Conv2d(1, 1, kernel_size=1)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )       

    def forward(self, x):
        x_conv = self.region_conv(x.unsqueeze(1)) 
        x_conv = x_conv.squeeze(1)  
        return self.mlp(x_conv)  

In [14]:
class RegionEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embd_dim, num_heads, num_layers):
        super().__init__()
        self.reg_embd = RegionEmbedder(input_dim, hidden_dim, embd_dim)
        self.mhsa_layers = nn.ModuleList([MHSA(embd_dim, num_heads) for _ in range(num_layers)])

    def forward(self, x):
        x_reg = self.reg_embd(x)
        x_in = x_reg
        attn_weights_all = []
        for mhsa in self.mhsa_layers:
            x_in, attn_weights = mhsa(x_in)
            attn_weights_all.append(attn_weights)
        
        return x_reg + x_in, torch.stack(attn_weights_all).permute(1, 0, 2, 3, 4)

In [15]:
class SubNetworkEncoder(nn.Module):
    def __init__(self, shapes, hidden_dim, embd_dim, num_heads, num_layers):
        super().__init__()
        self.embd_dim = embd_dim
        self.mlps = [SubnetworkEmbedder(i, hidden_dim, embd_dim) for i in shapes]
        self.mhsa_layers = nn.ModuleList([MHSA(embd_dim, num_heads) for _ in range(num_layers)])

    def forward(self, x):
        batch_size = x[0].shape[0]
        x = torch.stack([mlp(f) for mlp, f in zip(self.mlps, x)], dim=1)
        attn_weights_all = []
        for mhsa in self.mhsa_layers:
            x, attn_weights = mhsa(x)
            attn_weights_all.append(attn_weights)

        return x, torch.stack(attn_weights_all).permute(1, 0, 2, 3, 4)

In [16]:
model = SubNetworkEncoder(shapes, 256, 128, 8, 4)
a = next(iter(train_loader))[1]
model(a)[1].shape

torch.Size([16, 4, 8, 8, 8])

In [17]:
b = next(iter(train_loader))[0]
model1 = RegionEncoder(185, 256, 128, 8, 4)
model1(b)[1].shape

torch.Size([16, 4, 8, 185, 185])

In [18]:
class StepOne(nn.Module):
    def __init__(self, input_dim, hidden_dim, embd_dim, num_heads, num_layers):
        super().__init__()
        self.reg_enc = RegionEncoder(input_dim, hidden_dim, embd_dim, num_heads, num_layers)
        self.subnet_enc = SubNetworkEncoder(shapes, hidden_dim, embd_dim, num_heads, num_layers)
        self.layer_norm = nn.LayerNorm(embd_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embd_dim, hidden_dim),  
            nn.ReLU(),             
            nn.Linear(hidden_dim, embd_dim)    
        )
        self.region_start_indices = list(dataset.region_start_indices.values()) + [185]

    def subNetworkAttendRegions(self, subnet_attn_map, region_attn_map):
        region_to_subnet = torch.zeros(185, dtype=torch.long)
        for subnet_id in range(7):
            start = self.region_start_indices[subnet_id]
            end = self.region_start_indices[subnet_id + 1]
            region_to_subnet[start:end] = subnet_id  
        subnet_i = region_to_subnet.view(-1, 1).expand(185, 185) 
        subnet_j = region_to_subnet.view(1, -1).expand(185, 185)  

        mask = subnet_i != subnet_j  

        attn_multiplier = subnet_attn_map[:, :, :, subnet_i, subnet_j]  
        attn_multiplier = attn_multiplier * mask

        return region_attn_map * attn_multiplier 
    
    def sinkhorn(self, attn, n_iters=5, eps=1e-6):
        attn = attn + eps  
        for _ in range(n_iters):
            attn = attn / attn.sum(dim=-1, keepdim=True)
            attn = attn / attn.sum(dim=-2, keepdim=True)
        return attn
    
    def forward(self, x):
        x0 = self.reg_enc(x[0])
        x1 = self.subnet_enc(x[1])
        o = torch.cat((x0[0], x1[0]), dim=1)
        o_norm = self.layer_norm(o)
        o_norm = self.mlp(o)
        o = o + o_norm
        o_reg = o[:, :186, :]
        o_sub = o[:, 186:, :]
        adj_matrix = self.subNetworkAttendRegions(x1[1], x0[1])
        adj_matrix = self.sinkhorn(adj_matrix)
        return o_reg, o_sub, adj_matrix

In [19]:
model = StepOne(185, 256, 128, 16, 4)
a = model(next(iter(train_loader)))[2]
a.shape

torch.Size([16, 4, 16, 185, 185])

# HGCN part now

In [20]:
class HGCN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # Create weight matrices for each layer and head - just for regions (since we've alreay weigted them)
        self.W_reg = nn.ModuleList([
            nn.ModuleList([
                nn.Linear(input_dim, output_dim) for _ in range(16)  # 16 heads
            ]) for _ in range(4)  # 4 layers
        ])
        
        self.activation = nn.ReLU()
    
    def forward(self, features, attention_maps):
        batch_size, num_layers, num_heads, num_nodes, _ = attention_maps.shape
        all_outputs = []
        
        # go layer by layer
        for l in range(num_layers):
            layer_outputs = []
            # head by head
            for h in range(num_heads):
                # Get attention map for THIS layer and head
                A_l_c = attention_maps[:, l, h]
                
                # Transform feature with layer and head specific weights
                W_l_c = self.W_reg[l][h]
                transformed_features = W_l_c(features)  # [batch_size, num_nodes, output_dim]
                
                # graph convolution like in paper
                O_l_c = torch.bmm(A_l_c, transformed_features)  # [batch_size, num_nodes, output_dim]
                O_l_c = self.activation(O_l_c)
                
                layer_outputs.append(O_l_c)
            
            # Concatenate outputs from all heads for THIS layer
            layer_output = torch.cat(layer_outputs, dim=2)  # [batch_size, num_nodes, output_dim*num_heads]
            all_outputs.append(layer_output)
        
        # concatenate outputs from ALL layers
        # Z = ∥ᴸₗ₌₁(∥ᶜ𝒄₌₁ O̅ˡ'ᶜ)
        final_output = torch.cat(all_outputs, dim=2)  # [batch_size, num_nodes, output_dim*num_heads*num_layers]
        
        return final_output

In [21]:
class StepOneWithHGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, embd_dim, num_heads, num_layers, num_classes=2):
        super().__init__()
        # Copy paste
        self.reg_enc = RegionEncoder(input_dim, hidden_dim, embd_dim, num_heads, num_layers)
        self.subnet_enc = SubNetworkEncoder(shapes, hidden_dim, embd_dim, num_heads, num_layers)
        self.layer_norm = nn.LayerNorm(embd_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embd_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embd_dim)
        )
        self.region_start_indices = list(dataset.region_start_indices.values()) + [185]
        
        # Adding HGCN
        # For each head, we reduce dimension to embd_dim/num_heads
        self.hgcn = HGCN(input_dim=embd_dim, output_dim=embd_dim//num_heads)
        
        # Classifier for final prediction
        hgcn_output_dim = (embd_dim//num_heads) * num_heads * num_layers
        self.classifier = nn.Sequential(
            nn.Linear(hgcn_output_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def subNetworkAttendRegions(self, subnet_attn_map, region_attn_map):
        region_to_subnet = torch.zeros(185, dtype=torch.long)
        for subnet_id in range(7):
            start = self.region_start_indices[subnet_id]
            end = self.region_start_indices[subnet_id + 1]
            region_to_subnet[start:end] = subnet_id
        subnet_i = region_to_subnet.view(-1, 1).expand(185, 185)
        subnet_j = region_to_subnet.view(1, -1).expand(185, 185)
        mask = subnet_i != subnet_j
        attn_multiplier = subnet_attn_map[:, :, :, subnet_i, subnet_j]
        attn_multiplier = attn_multiplier * mask
        return region_attn_map * attn_multiplier
    
    def sinkhorn(self, attn, n_iters=5, eps=1e-6):
        attn = attn + eps
        for _ in range(n_iters):
            attn = attn / attn.sum(dim=-1, keepdim=True)
            attn = attn / attn.sum(dim=-2, keepdim=True)
        return attn
    
    def forward(self, x):
        # Step One processing
        x0, region_attn = self.reg_enc(x[0])
        x1, subnet_attn = self.subnet_enc(x[1])
        
        o = torch.cat((x0, x1), dim=1)
        o_norm = self.layer_norm(o)
        o_norm = self.mlp(o_norm)
        o = o + o_norm
        
        o_reg = o[:, :185, :]  # First 185 nodes are regions
        o_sub = o[:, 185:, :]  # Remaining nodes are subnetworks
        
        # Process attention maps to create the combined attention 
        # with shape [batch_size, num_layers (4), num_heads (16), 185, 185]
        combined_attn = self.subNetworkAttendRegions(subnet_attn, region_attn)
        combined_attn = self.sinkhorn(combined_attn)
        
        # Pass through HGCN - only process the region features with the combined attention
        hgcn_output = self.hgcn(o_reg, combined_attn)
        
        # Global average pooling for classification
        pooled_output = hgcn_output.mean(dim=1)  # [batch_size, output_dim*num_heads*num_layers]
        
        # Final classif
        logits = self.classifier(pooled_output)
        
        return logits, o_reg, o_sub, combined_attn, hgcn_output

In [22]:
for param in model.parameters():
    print(param.device)

cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu


In [24]:
model = StepOneWithHGCN(
    input_dim=185,
    hidden_dim=256,
    embd_dim=128,
    num_heads=16,
    num_layers=4,
    num_classes=2
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # For classification

num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (region_data, subnetwork_data, labels) in enumerate(train_loader):
        region_data = region_data.to(device)
        subnetwork_data = [subnet.to(device) for subnet in subnetwork_data] 
        labels = labels.to(device)
        # I MOVED EVERYTHING ALREADY

        
        optimizer.zero_grad()
        
        # Forward pass
        x = (region_data, subnetwork_data)
        logits, _, _, _, _ = model(x)
        
        loss = criterion(logits, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], '
                  f'Loss: {running_loss/10:.4f}, Accuracy: {100*correct/total:.2f}%')
            running_loss = 0.0

torch.save(model.state_dict(), 'brain_network_model.pth')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)